In [ ]:
## Installing API packages in order to retrieve data
!pip install nasdaq-data-link
!pip install fredapi

## Import the Packages and Set Your API Keys

In [ ]:
import pandas as pd
import numpy as np
import requests
import warnings
import nasdaqdatalink
from fredapi import Fred
import os
import time

fred_api_key = os.environ["FRED_API_KEY"]

fred = Fred(api_key=fred_api_key)

warnings.filterwarnings("ignore")

nasdaqdatalink.ApiConfig.api_key = os.environ['NASDAQ_API']


## Retrieve Zillow's Indicators & Regions into a Pandas Dataframe

In [ ]:
# Creating DataFrames of Zillow's available indicators and regions so that they can be queried and called

Indicators = nasdaqdatalink.get_table('ZILLOW/INDICATORS')
Regions = nasdaqdatalink.get_table('ZILLOW/REGIONS')

## Read Realtor.com's Inventory Hotness Metrics into a Pandas Dataframe

In [ ]:
# Here we read realtor.com's Inventory Hotness research data for metropolitan areas

rdcInventoryHotness = pd.read_csv('RDC_InventoryHotness_Metro.csv')


## Retrieve FRED's Per Capita Personal Income for Each Region

In [ ]:
# Retrieving FRED's personal income series which are available through the 'search_by_release' API call

per_cap_personal_income = fred.search_by_release(175, limit=377, order_by='popularity', sort_order='desc')

per_cap_personal_income.info()
per_cap_personal_income[['id', 'title', 'popularity']].head()

In [ ]:
# Writing a function which will retrieve per capita personal income time series by the id number and add the time series into a pandas DataFrame
import time

def make_api_call(series):

    data = {}

    for value in series:
        data[value] = fred.get_series(value)
        time.sleep(1)
    
    data = pd.DataFrame(data)

    return data

In [ ]:
# Retrieving each region's Per Capita Personal Income time series data
# regional_income_df = make_api_call(per_cap_personal_income['id'])

# Feeding the retrieved data into a csv so that there is no need to repeat the API call if the notebook is closed or revisited at another time
# regional_income_df.to_csv('regional_income_df.csv', encoding='utf-8', index=False)

## Retrieve the National Average for Comparison

In [ ]:
zillow_regions = nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type='state')

all_states_ids = zillow_regions['region_id']

all_states_index = nasdaqdatalink.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id=all_states_ids)

national_index = all_states_index.groupby('date').agg({'value': 'sum'})

In [ ]:
print(national_index)

## Metric 1: Realtor.com's Hotness Score

Query the Inventory Hotness dataset for the backtest period starting date. In this case, April 2018.

In [ ]:
SELECT *
FROM rdcInventoryHotness
WHERE month_date_yyyymm LIKE '201804';

The previous query provides a dataframe of regions ranked by Hotness Score. On the dataframe we use a for loop to iterate through each region and call the Zillow API for the historical price data. The data is fed into a new dataframe with each regions time series data.

In [ ]:
# # Get the unique values of the column cbsa_title
# unique_values = df2['cbsa_title'].unique()

# # Create an empty list to store the results of the SQL query
# comparing_all_regions = national_index
# # Loop through the unique values and execute the SQL query for each value
# for region_value in unique_values:
#     list_of_column = pd.DataFrame()
#     # Replace VALUE with the current value in the SQL query
#     list_of_column = Regions[Regions.region.str.contains(f'{region_value}')]
#     if len(list_of_column) > 0:
#         each_region_id = list_of_column['region_id']
#     else:
#         continue
#     api_results = nasdaqdatalink.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id=each_region_id)
#     if not isinstance(api_results, pd.DataFrame):
#         continue
#     new_dataframe = api_results.groupby('date').agg({'value': 'sum'})
#     final_dataframe = pd.DataFrame()
#     final_dataframe[str(region_value)] = round(new_dataframe['value'], 2)
#     comparing_all_regions = pd.merge(comparing_all_regions, final_dataframe, how='outer', on='date')

# Merge results_of_second_query into an existing database
print(comparing_all_regions.info())

In [ ]:
# Save results to csv so that we don't need to repeat the query when returning to the notebook
# all_regions_hotnessscore_ranked = comparing_all_regions.to_csv('all_regions_hotness_score_ranked.csv')

Using a for loop on the new dataframe, the compound annual growth rate(CAGR) can be calculated for each region. The dictionary of CAGRs can then be mapped onto the original dataframe of the backtest period's start month for further analysis.

In [ ]:

comparing_all_regions = comparing_all_regions.set_index('date')

cagr = {}

for column in comparing_all_regions.columns:
    try:
        start_value = comparing_all_regions.at['2018-04-30', column]
        end_value = comparing_all_regions.at['2023-04-30', column]
        cagr[column] = (end_value/start_value)**(1/5) - 1
    except KeyError:
        print(f"Skipping {column} due to missing data")

print(cagr)


In [ ]:
df2['cagr'] = df2['cbsa_title'].map(cagr, na_action='ignore')


df2.to_csv('metric1_cagr.csv')

## Metric 2: The Demand-Supply Ratio

For this metric realtor.com's Inventory Hotness data is used to create this new metric measuring demand in comparison to supply.

The first step is to simply append a new column which divides the demand score by the supply score for each row.

In [ ]:
rdcInventoryHotness['DemandSupplyRatio'] = rdcInventoryHotness['demand_score']/rdcInventoryHotness['supply_score']

Once again, the dataframe is queried for the start of the backtest period, ordering the results by the Demand Supply Ratio, and excluding rows with null values in the demand supply ratio calculation.

In [ ]:
SELECT *
FROM rdcInventoryHotness
WHERE DemandSupplyRatio NOT NULL
AND month_date_yyyymm LIKE '201804'
AND supply_score NOT NULL
AND demand_score NOT NULL
ORDER BY DemandSupplyRatio DESC;

The ranked list of regions is used in a for loop to retrieve historical price data for each region, returning a new dataframe.

In [ ]:
# Get the unique values of the column cbsa_title
unique_values = df4['cbsa_title'].unique()

# Create an empty list to store the results of the SQL query
comparing_all_regions2 = national_index
# Loop through the unique values and execute the SQL query for each value
for region_value in unique_values:
    list_of_column = pd.DataFrame()
    # Replace VALUE with the current value in the SQL query
    list_of_column = Regions[Regions.region.str.contains(f'{region_value}')]
    if len(list_of_column) > 0:
        each_region_id = list_of_column['region_id']
    else:
        continue
    api_results = nasdaqdatalink.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id=each_region_id)
    if not isinstance(api_results, pd.DataFrame):
        continue
    new_dataframe = api_results.groupby('date').agg({'value': 'sum'})
    final_dataframe = pd.DataFrame()
    final_dataframe[str(region_value)] = round(new_dataframe['value'], 2)
    comparing_all_regions2 = pd.merge(comparing_all_regions2, final_dataframe, how='outer', on='date')


# Merge results_of_second_query into an existing database
print(comparing_all_regions2.info())

In [ ]:
# Save the results so that further API calls are not needed when we return to the notebook.
comparing_all_regions2.to_csv('every_region_index_201804.csv')

An empty dictionary is created and a for loop to calculate the CAGR for each region will fill the dictionary. The resulting dictionary is then mapped onto the backtest beginning dataframe.

In [ ]:
# comparing_all_regions2 = comparing_all_regions2.set_index('date')

cagr = {}
for column in comparing_all_regions2.columns:
    try:
        start_value = comparing_all_regions2.at['2018-04-30', column]
        end_value = comparing_all_regions2.at['2023-04-30', column]
        cagr[column] = (end_value/start_value)**(1/5) - 1
    except KeyError:
        print(f"Skipping {column} due to missing data")
        
print(cagr)

In [ ]:
df4['cagr'] = df4['cbsa_title'].map(cagr, na_action='ignore')

In [ ]:
df4.to_csv('metric2_cagr.csv')


## Metric 3: Per Capita Personal Income

To start, the iloc method is used to create a series with every region from 2016, the earliest period we can backtest the metric on.

In [ ]:
regional_income_df = pd.read_csv('regional_income_df.csv')

year_2016 = regional_income_df.iloc[47]

year_2016.info()

The series is processed and cleaned

In [ ]:
# The index is reset since the index contains the id values for each region, this way we can call the region 'id'.
year_2016 = year_2016.reset_index()

# The column name is changed from 47 to 'year2016' for easier indexing. A new column is created copying the values and the column 47 is then dropped.
year_2016['year2016'] = year_2016.iloc[:, 1].values
year_2016 = year_2016.drop(year_2016.columns[1], axis=1)
year_2016.info()

We sort the data, ranking the regions by highest income.

In [ ]:
year2016incomes_sorted = year_2016.sort_values(by=year_2016.columns[1], ascending=False)

With the ranked list of regions we query the dataframe of release information to only include Per Capita Personal Income, ranked by highest income.

In [ ]:
SELECT *
FROM per_cap_personal_income
WHERE id IN (SELECT index
FROM year2016incomes_sorted
WHERE index LIKE '%PCPI%'
ORDER BY year2016 DESC)

Using the title column for each region we can call the Zillow API to retrive home price values. The title column is cleaned a for loop is used to gather every region's home price data into a dataframe.

In [ ]:
areas = df7['title'].apply(lambda x: x.replace('Per Capita Personal Income in ', ''))

areas = areas.apply(lambda x: x.replace(' (MSA)', ''))


print(areas)

# Create an empty list to store the results of the SQL query
comparison_dataframe3 = national_index
# Loop through the unique values and execute the SQL query for each value
for region_value in areas:
    list_of_column = pd.DataFrame()
    
    list_of_column = Regions[Regions.region.str.contains(f'{region_value}')]
    if len(list_of_column) > 0:
        each_region_id = list_of_column['region_id']
    else:
        continue
    api_results = nasdaqdatalink.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id=each_region_id)
    if not isinstance(api_results, pd.DataFrame):
        continue
    new_dataframe = api_results.groupby('date').agg({'value': 'sum'})
    final_dataframe = pd.DataFrame()
    final_dataframe[str(region_value)] = round(new_dataframe['value'], 2)
    comparison_dataframe3 = pd.merge(comparison_dataframe3, final_dataframe, how='outer', on='date')

# Merge results_of_second_query into an existing database
print(comparison_dataframe3.info())

CAGR is calculated for each region.

In [ ]:
# comparison_dataframe3 = comparison_dataframe3.set_index('date')

cagr = {}
for column in comparison_dataframe3.columns:
    try:
        start_value = comparison_dataframe3.at['2018-04-30', column]
        end_value = comparison_dataframe3.at['2023-04-30', column]
        cagr[column] = (end_value/start_value)**(1/5) - 1
    except KeyError:
        print(f"Skipping {column} due to missing data")
        
print(cagr)

In [ ]:
map_table = df7[['id', 'title']]
map_table = map_table.set_index('id')

map_table['title'] = map_table['title'].apply(lambda x: x.replace('Per Capita Personal Income in ', ''))

map_table['title'] = map_table['title'].apply(lambda x: x.replace(' (MSA)', ''))

year2016incomes_sorted = year2016incomes_sorted.rename(columns={'index': 'id'})

correlation_table = pd.merge(year2016incomes_sorted, map_table, on='id')
correlation_table['cagr'] = correlation_table['title'].map(cagr, na_action='ignore')

In [ ]:
correlation_table.to_csv('metric3_cagr.csv')